In [ ]:
from google.cloud import bigquery
import pandas as pd

def send_data_to_bq(bigquery_client, TABLE_NAME, clean_dataframe):
    '''
        Drop existing data in the BQ table for the months
        that come in the dataframe
    '''
    # Drop data for corresponding months
    dropped = drop_previous_data(bigquery_client, TABLE_NAME, clean_dataframe)

    # If dropped successfully, insert new values
    if dropped:
        # List of columns in the dataframe
        columns = ["Conversation_id", "ANI", "Fecha", "Segmento", "Rut_empresa", "Tipo_de_cliente", "EPA", "Resolucion", "Transferidas_a_ejecutivo", "Event", "Autoatencion", "Transferencia_auto","error_api","FullStackB2B"]
        insert_data_batched(bigquery_client, TABLE_NAME, clean_dataframe, columns, batch_size=1000)

def drop_previous_data(bigquery_client, TABLE_NAME, clean_dataframe):
    '''
        Delete existing data in the table for the months
        to be uploaded
    '''
    # Detect which months (identified as year-month) are present in the dataframe
    # List of unique year-month combinations
    unique_year_month = tuple(clean_dataframe['Fecha'].dt.to_period('M').unique().astype(str).tolist())

    if len(unique_year_month) == 1:
        unique_year_month = str(unique_year_month)[:-2] + str(unique_year_month)[-1:]

    # Build query to delete values from the table
    QUERY = (
        f"DELETE FROM {TABLE_NAME} "
        f"WHERE FORMAT_DATE('%Y-%m', Fecha) IN {unique_year_month}"
    )

    # Delete from the table the data for the months present in the dataframe
    try:
        query_job = bigquery_client.query(QUERY)
        query_job.result()  # Waits for job to complete.
        return True
    except Exception as err:
        print("Error deleting previous values from the table")
        print(err)
        return False

def insert_data_batched(bigquery_client, TABLE_NAME, clean_dataframe, columns, batch_size):
    '''
        Insert values from the dataframe into BQ in batches,
        keep track of successes and errors
    '''
    inserts = 0
    errors = 0

    column_list = ', '.join(columns)

    # Iterate over the dataframe to build values to insert
    for start_index in range(0, len(clean_dataframe), batch_size):
        end_index = start_index + batch_size
        batch_dataframe = clean_dataframe.iloc[start_index:end_index]

        values_to_insert = []
        for index, row in batch_dataframe.iterrows():
            formatted_row = tuple(row.apply(lambda x: x.strftime('%Y-%m-%d') if isinstance(x, pd.Timestamp) else x))
            values_to_insert.append(formatted_row)

        # Create Query
        values_to_insert_str = ', '.join(map(str, values_to_insert))
        QUERY = (
            f'INSERT INTO {TABLE_NAME} ({column_list}) '
            f'VALUES {values_to_insert_str}'
        )

        try:
            # Call BigQuery API
            query_job = bigquery_client.query(QUERY)
            results = query_job.result()  # Waits for job to complete.
            inserts += len(batch_dataframe)
        except Exception as err:
            print("Error inserting values:")
            print(err)
            errors += len(batch_dataframe)

    print(f"Loading completed for table {TABLE_NAME}. Total inserts: {inserts}, Total errors: {errors}")